# GSCDB138 Dataset Analysis

This notebook analyzes computational chemistry functional performance across the GSCDB138 benchmark database. The analysis includes:

- Processing reaction energies from molecular energies
- Converting finite difference data to physical properties (dipoles, polarizabilities, frequencies)
- Computing error metrics (RMSE, MAE, relative errors) for different functionals
- Statistical analysis across different property types

**Input requirement:** `Molecule_Energies.xlsx` file containing:
- **Column names:** Different computational methods/functionals
- **Row names:** Individual molecules required in GSCDB138

**Output Files Generated:**
1. **Reaction_Energies.xlsx**: Processed reaction energies from molecular data
2. **Results.xlsx**: Complete values and errors in the common units for all functionals
3. **Errors_per_set.xlsx**: Error metrics (RMSE, MAE, MSE, specialized metrics) by dataset
4. **Relative_metric_per_set.xlsx**: Normalized errors for cross-dataset comparison
5. **Statistical_errors.xlsx**: Final functional performance ranking

## Import Libraries and Setup

Import required libraries and configure pandas display options for better output visualization.

In [1]:
# Standard library imports
from glob import glob
import shutil
import os, sys, math

# Data analysis libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

# Configure pandas display options for better visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 5000)

## Step 1: Load Data and Calculate Reaction Energies

The following section processes stoichiometric data to calculate reaction energies from individual molecular energies.

In [2]:
# Load dataset information and molecular energies
df_func = pd.read_excel("../Info/DatasetEval.xlsx", header=0, index_col=0)
df_func = df_func[~df_func["Dataset"].isin(["SC74", "OEEFD"])]
df_energies = pd.read_excel("Molecule_Energies.xlsx", index_col=0, header=0)
funcs = df_energies.columns.tolist()

# Convert reference energies from Hartree to kcal/mol
df_func["Reference"] *= 627.50947406     

# Calculate reaction energies for each functional
print("Calculating reaction energies...")
for idx, row in tqdm(df_func.iterrows()):
    reactions = row["Stoichiometry"].split(",")  # Parse stoichiometry string
    ref_energy = row["Reference"]
    num_species = len(reactions) // 2  # Each species has coefficient and name
    
    for func in funcs:
        reac_energy = 0
        error = False
        
        # Sum up contributions from all species in the reaction
        for i in range(num_species):
            stoi = float(reactions[2*i])      # Stoichiometric coefficient
            specy = reactions[2*i+1]          # Species name
            energy = df_energies.loc[specy, func]
            reac_energy += stoi * energy
            
        if error:
            continue
        
        # Convert reaction energy from Hartree to kcal/mol
        df_func.at[idx, func] = reac_energy * 627.50947406

# Clean up dataframe by removing stoichiometry column
df_func.drop(columns=["Stoichiometry"], inplace=True)
print("Available functionals:", df_func.columns.tolist())

Calculating reaction energies...


8377it [00:01, 4255.43it/s]

Available functionals: ['Dataset', 'Reference', 'wB97M2', 'B3LYP', 'BMK', 'CAMB3LYP', 'CF22D', 'M052X', 'M062X', 'M08HX', 'MN15', 'PBE0', 'PW6B95', 'SOGGA11X', 'r2SCAN0', 'wB97M-V', 'wB97X-V', 'revDSD-PBEP86-D4']


## Step 2: Data Validation and Quality Control

Check for missing data (NaN values) and ensure we only analyze datasets with complete functional coverage.

In [3]:
# Check for NaN or None values by dataset and functional
incomplete_datasets = []

for dataset, df_dataset in df_func.groupby("Dataset"):
    empty_funcs = []
    
    # Check each functional column for NaN/None values
    for func in funcs:
        nan_count = df_dataset[func].isnull().sum()
        if nan_count > 0:
            empty_funcs.append(func)
    
    if empty_funcs:
        print(f"Dataset '{dataset}' has NaN values in functionals: {', '.join(empty_funcs)}")
        incomplete_datasets.append(dataset)

print(f"Incomplete datasets: {incomplete_datasets}")

# Filter to remove datasets with incomplete data
df_func = df_func[~df_func["Dataset"].isin(incomplete_datasets)]

# Save processed reaction energies
print(f"Saved reaction energies for {len(df_func)} data points")
df_func.to_excel("Reaction_Energies.xlsx", index=True, header=True)


Incomplete datasets: []
Saved reaction energies for 8377 data points


## Step 3: Unit Conversion Functions

Define functions to convert finite difference energies to physical properties:
- **Dipole moments**: From energy differences to Debye units
- **Polarizabilities**: From energy differences to Å³ units  
- **Vibrational frequencies**: From force constants to cm⁻¹ units

Data conversion is performed for specific datasets:
- **Dip146**: Dipole moments (step size: 0.0001)
- **Pol130**: Polarizabilities (variable step sizes from FD_stepsize.xlsx)
- **HR46, T144**: Polarizabilities (step size: 0.004)
- **V30**: Vibrational frequencies (variable step sizes from FD_stepsize.xlsx)

In [4]:
def convert_to_dipole(energy, step_size=0.0001, input_in_kcal=True, output_in_debye=True):
    """
    Convert finite difference energy to dipole moment.
    
    Parameters:
    -----------
    energy : float
        Energy difference from finite difference calculation
    step_size : float, default=0.0001
        Step size used in finite difference calculation
    input_in_kcal : bool, default=True
        Whether input energy is in kcal/mol (vs Hartree)
    output_in_debye : bool, default=True
        Whether to output in Debye units (vs atomic units)
    
    Returns:
    --------
    float : Dipole moment in specified units
    """
    au_to_debye = 2.541746
    output = energy
    
    if input_in_kcal:
        output /= 627.50947406  # Convert kcal/mol to Hartree
    
    output /= 2 * step_size  # Convert to dipole moment in atomic units
    
    if output_in_debye:
        output *= au_to_debye  # Convert to Debye
    
    return output


def convert_to_polarizability(energy, step_size=0.01, input_in_kcal=True, output_in_A3=True):
    """
    Convert finite difference energy to polarizability.
    
    Parameters:
    -----------
    energy : float
        Energy difference from finite difference calculation
    step_size : float, default=0.01
        Step size used in finite difference calculation
    input_in_kcal : bool, default=True
        Whether input energy is in kcal/mol (vs Hartree)
    output_in_A3 : bool, default=True
        Whether to output in Å³ (vs atomic units)
    
    Returns:
    --------
    float : Polarizability in specified units
    """
    au_to_bohr3 = 0.529177 ** 3  # Conversion factor from atomic units to Bohr³
    output = energy
    
    if input_in_kcal:
        output /= 627.50947406  # Convert kcal/mol to Hartree
    
    output /= step_size**2  # Convert to polarizability in atomic units
    
    if output_in_A3:
        output *= au_to_bohr3  # Convert to Å³
    
    return output


def convert_to_frequency(energy, step_size=0.01, input_in_kcal=True, output_in_cm=True):
    """
    Convert finite difference energy to vibrational frequency.
    
    Parameters:
    -----------
    energy : float
        Energy difference from finite difference calculation (force constant)
    step_size : float, default=0.01
        Step size used in finite difference calculation
    input_in_kcal : bool, default=True
        Whether input energy is in kcal/mol (vs Hartree)
    output_in_cm : bool, default=True
        Whether to output in cm⁻¹ (vs atomic units)
    
    Returns:
    --------
    float : Frequency in specified units (preserves sign for imaginary frequencies)
    """
    constant_cm_to_hartree = 4.55633528e-6
    convert_force_constant_to_au = 1 / (1.66053907e-27/9.10938370e-31 * (1/0.529177211)**2)
    
    fc = energy
    if input_in_kcal:
        fc /= 627.50947406  # Convert kcal/mol to Hartree
    
    signature = np.sign(fc)  # Preserve sign for imaginary frequencies
    fc = np.abs(fc)  # Take absolute value for frequency calculation
    fc /= step_size**2  # Calculate force constant in Hartree/Å²
    fc *= convert_force_constant_to_au  # Convert to atomic units
    freq = np.sqrt(fc)  # Calculate frequency in atomic units
    
    if output_in_cm:
        freq /= constant_cm_to_hartree  # Convert to cm⁻¹
    
    freq *= signature  # Restore sign
    return freq

In [5]:
# Load processed reaction energies and step size information
df_func = pd.read_excel("Reaction_Energies.xlsx", index_col=0, header=0)
df_stepsizes = pd.read_excel("../Info/FD_stepsize.xlsx", header=0, index_col=0)

# Get all functional columns (skip "Dataset" but also process "Reference")
funcs = df_func.columns[1:]

# Convert Dip146 dataset: Energy differences → Dipole moments (Debye)
print("Converting Dip146 to dipole moments...")
dip146_index = df_func.index[df_func["Dataset"] == "Dip146"].to_list()
for idx in dip146_index:
    for func in funcs:
        df_func.at[idx, func] = convert_to_dipole(
            df_func.at[idx, func], step_size=0.0001, 
            input_in_kcal=True, output_in_debye=True)

# Convert Pol130 dataset: Energy differences → Polarizabilities (Å³)
print("Converting Pol130 to polarizabilities...")
pol130_index = df_func.index[df_func["Dataset"] == "Pol130"].to_list()
for idx in pol130_index:
    step_size = df_stepsizes.at[idx, "fd step"]
    for func in funcs:
        df_func.at[idx, func] = convert_to_polarizability(
            df_func.at[idx, func], step_size=step_size,
            input_in_kcal=True, output_in_A3=True)

# Convert HR46 and T144 datasets: Energy differences → Polarizabilities (Å³)
print("Converting HR46 and T144 to polarizabilities...")
hr46_and_t144_index = df_func.index[df_func["Dataset"].isin(["HR46", "T144"])].to_list()
for idx in hr46_and_t144_index:
    for func in funcs:
        df_func.at[idx, func] = convert_to_polarizability(
            df_func.at[idx, func], step_size=0.004,
            input_in_kcal=True, output_in_A3=True)

# Convert V30 dataset: Force constants → Vibrational frequencies (cm⁻¹)
print("Converting V30 to vibrational frequencies...")
v30_index = df_func.index[df_func["Dataset"] == "V30"].to_list()
for idx in v30_index:
    step_size = df_stepsizes.at[idx, "fd step"]
    for func in funcs:
        df_func.at[idx, func] = convert_to_frequency(
            df_func.at[idx, func], step_size=step_size,
            input_in_kcal=True, output_in_cm=True)


Converting Dip146 to dipole moments...
Converting Pol130 to polarizabilities...
Converting HR46 and T144 to polarizabilities...
Converting V30 to vibrational frequencies...


## Step 4: Save Results

In [6]:
# Calculate absolute errors for each functional
df_errors = df_func.copy()

# Compute errors as (calculated - reference) for functional columns
for func in df_errors.columns[2:]:  # Skip "Dataset" and "Reference" columns
    df_errors[func] = df_func[func] - df_func["Reference"]

# Save both values and errors to Excel file
print("Saving results to Results.xlsx...")
with pd.ExcelWriter("Results.xlsx") as writer:
    df_func.to_excel(writer, index=True, sheet_name="values")
    df_errors.to_excel(writer, index=True, sheet_name="errors")


Saving results to Results.xlsx...


## Step 5: Calculate Errors

Compute root-mean-square error (RMSE), mean absolute error (MAE), mean signed error (MSE), and our metric errors (defined in the benchmark paper) for each functional.

In [10]:
# Load error data and supporting information
df_error = pd.read_excel("Results.xlsx", index_col=0, header=0, sheet_name="errors")
TMC34_weight = pd.read_excel("../Info/DatasetEval.xlsx", index_col=0, header=0, sheet_name="TMC34")
O24x5_weight = pd.read_excel("../Info/DatasetEval.xlsx", index_col=0, header=0, sheet_name="O24x5")
df_dataset_info = pd.read_excel("../Info/Datasets.xlsx", header=0, index_col=0, sheet_name="Sheet1")

# Get list of functionals (exclude Reference and Dataset columns)
funcs = df_error.columns.tolist()
funcs.remove("Reference")
funcs.remove("Dataset")

# Initialize dictionaries to store different error metrics
df_rmse_sets = {}
df_mae_sets = {}
df_mse_sets = {}
df_metric_set = {}

print("Calculating error metrics by dataset...")

# Calculate error metrics for each dataset
for dataset, df_dataset in df_error.groupby("Dataset"):
    # Initialize dataset entries with datatype information
    datatype = df_dataset_info.loc[dataset, "Datatype"]
    df_rmse_sets[dataset] = {"Datatype": datatype}
    df_mae_sets[dataset] = {"Datatype": datatype}
    df_metric_set[dataset] = {"Datatype": datatype}
    df_mse_sets[dataset] = {"Datatype": datatype}
    
    # Calculate standard error metrics for each functional
    for func in funcs:
        if func in df_error.columns:
            df_rmse_sets[dataset][func] = np.sqrt(np.mean(df_dataset[func]**2))  # Root Mean Square Error
            df_mae_sets[dataset][func] = np.mean(np.abs(df_dataset[func]))       # Mean Absolute Error
            df_mse_sets[dataset][func] = np.mean(df_dataset[func])               # Mean Signed Error
            df_metric_set[dataset][func] = df_mae_sets[dataset][func]            # Default metric is MAE
    
    # Apply specialized metrics for specific datasets
    # Mean Absolute Relative Error for polarizability and field-dependent datasets
    if dataset in ["Pol130", "HR46", "T144", "OEEF"]:
        for func in funcs:
            if func in df_error.columns:
                df_metric_set[dataset][func] = np.mean(np.abs(df_dataset[func] / df_dataset["Reference"]))
    
    # Weighted error for TMC datasets
    if dataset in ["TMD10", "MOR13", "TMB11"]:
        weight = TMC34_weight.loc[TMC34_weight["Dataset"] == dataset, "Weight"].values
        for func in funcs:
            if func in df_error.columns:
                weighted_errors = np.abs(df_dataset[func]) * weight[:-1]
                df_metric_set[dataset][func] = np.sum(weighted_errors) + weight[-1]
    
    # Weighted error for O24x5 datasets
    if dataset in ["O24", "O24x4"]: # calculate weighted absolute error
        weight = O24x5_weight.loc[O24x5_weight["Dataset"] == dataset, "Weight"].values
        for func in funcs:
            if func in df_error.columns:
                weighted_errors = np.abs(df_dataset[func]) * weight
                df_metric_set[dataset][func] = np.sum(weighted_errors)

    # Regularized MAE for dipole moments (normalized by max(|reference|, 1))
    if dataset == "Dip146":
        for func in funcs:
            if func in df_error.columns:
                df_metric_set[dataset][func] = np.mean(
                    np.abs(df_dataset[func] / np.maximum(np.abs(df_dataset["Reference"]), 1))
                )

# Convert dictionaries to DataFrames and sort by datatype and dataset name
df_rmse_sets = pd.DataFrame(df_rmse_sets).T
df_rmse_sets.index.name = "Dataset"
df_rmse_sets = df_rmse_sets.sort_values(by=["Datatype", "Dataset"])

df_mae_sets = pd.DataFrame(df_mae_sets).T
df_mae_sets.index.name = "Dataset"
df_mae_sets = df_mae_sets.sort_values(by=["Datatype", "Dataset"])

df_metric_set = pd.DataFrame(df_metric_set).T
df_metric_set.index.name = "Dataset"
df_metric_set = df_metric_set.sort_values(by=["Datatype", "Dataset"])

df_mse_sets = pd.DataFrame(df_mse_sets).T
df_mse_sets.index.name = "Dataset"
df_mse_sets = df_mse_sets.sort_values(by=["Datatype", "Dataset"])

# Save all error metrics to Excel file
print("Saving error metrics to Errors_per_set.xlsx...")
with pd.ExcelWriter("Errors_per_set.xlsx") as writer:
    df_rmse_sets.to_excel(writer, index=True, sheet_name="RMSE")
    df_mae_sets.to_excel(writer, index=True, sheet_name="MAE")
    df_metric_set.to_excel(writer, index=True, sheet_name="Metric")
    df_mse_sets.to_excel(writer, index=True, sheet_name="MSE")

Calculating error metrics by dataset...
Saving error metrics to Errors_per_set.xlsx...


## Step 6: Statistical Analysis by Dataset

To establish a robust baseline for comparison, we define a “standard error” for each data set as the average of the second, third, and fourth lowest errors among all tested hybrid functionals.

Then we calculate normalized error ratio (NER), defined as the ratio between its error and the standard error for a given data set.

In [11]:
# Load metric data and standard reference errors
df_metric_set = pd.read_excel("Errors_per_set.xlsx", index_col=0, header=0, sheet_name="Metric")
df_standard = pd.read_excel("../Info/Standard_errors.xlsx", index_col=0, header=0)
standard_metric = df_standard["Metric"]

# Calculate relative errors by normalizing against standard reference values
df_errors_rel = df_metric_set.copy()

print("Calculating relative errors...")
for func in df_metric_set.columns:
    if func == "Datatype":
        continue
    if func in df_metric_set.columns:
        # Normalize each functional's errors by the corresponding standard error
        df_errors_rel[func] = df_metric_set[func] / standard_metric

# Save relative errors
df_errors_rel.to_excel("Relative_metric_per_set.xlsx", index=True, header=True)
print("Saved relative errors for cross-dataset comparison")

Calculating relative errors...
Saved relative errors for cross-dataset comparison


## Step 7: Generate Final Statistical Summary

Create overall performance rankings by calculating mean relative errors across all datasets and by property type. Functionals are ranked by overall performance.

In [12]:
# Load relative errors for final analysis
df_errors_rel = pd.read_excel("Relative_metric_per_set.xlsx", index_col=0, header=0)

# Initialize summary statistics DataFrame
df_errors = pd.DataFrame(columns=funcs)

# Calculate overall mean relative error across all datasets
df_errors.loc["Mean"] = df_errors_rel[funcs].mean(axis=0)

# Calculate mean relative error by property type (datatype)
print("Calculating statistics by property type...")
for datatype, df_errors_set_datatype in df_errors_rel.groupby("Datatype"):
    df_errors_set_datatype = df_errors_set_datatype.drop(columns=["Datatype"])
    df_errors.loc[f"Mean {datatype}"] = df_errors_set_datatype.mean(axis=0)

# Sort functionals by overall performance (ascending mean error)
sorted_funcs = sorted(funcs, key=lambda x: df_errors.loc["Mean", x])
df_errors = df_errors[sorted_funcs]

# Save final statistical summary
df_errors.to_excel("Statistical_errors.xlsx", index=True, header=True)

print("Analysis complete! Files generated:")
print("- Reaction_Energies.xlsx: Processed reaction energies")
print("- Results.xlsx: Values and errors for all functionals")
print("- Errors_per_set.xlsx: Error metrics by dataset")
print("- Relative_metric_per_set.xlsx: Normalized relative errors")
print("- Statistical_errors.xlsx: Final performance ranking")

# Display top 5 performing functionals
print(f"\nTop 5 performing functionals (lowest mean relative error):")
for i, func in enumerate(sorted_funcs[:5], 1):
    mean_error = df_errors.loc["Mean", func]
    print(f"{i}. {func}: {mean_error:.3f}")

Calculating statistics by property type...
Analysis complete! Files generated:
- Reaction_Energies.xlsx: Processed reaction energies
- Results.xlsx: Values and errors for all functionals
- Errors_per_set.xlsx: Error metrics by dataset
- Relative_metric_per_set.xlsx: Normalized relative errors
- Statistical_errors.xlsx: Final performance ranking

Top 5 performing functionals (lowest mean relative error):
1. wB97M2: 0.949
2. wB97M-V: 1.076
3. CF22D: 1.249
4. wB97X-V: 1.313
5. revDSD-PBEP86-D4: 1.522
